## 2.2 Few Shot Prompt Template

<div style="text-align: right"> Initial issue : 2025.04.28 </div>
<div style="text-align: right"> last update : 2025.04.28 </div>

- 시스템 프롬프트 없이 질문, 답변, 예시 반복후 질문을 입력해도 답변 잘함
- 할루시네이션도 줄일 수 있음

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from utils import langsmith, stream_response
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [3]:
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

### 1. FewShotPromptTemplate 생성하기

In [4]:
examples = [
    {
        "question": "스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인
""",
    },
    {
        "question": "네이버의 창립자는 언제 태어났나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일
""",
    },
    {
        "question": "율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 율곡 이이의 어머니는 누구인가요?
중간 답변: 율곡 이이의 어머니는 신사임당입니다.
추가 질문: 신사임당은 언제 태어났나요?
중간 답변: 신사임당은 1504년에 태어났습니다.
추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
최종 답변은: 연산군
""",
    },
    {
        "question": "올드보이와 기생충의 감독이 같은 나라 출신인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 올드보이의 감독은 누구인가요?
중간 답변: 올드보이의 감독은 박찬욱입니다.
추가 질문: 박찬욱은 어느 나라 출신인가요?
중간 답변: 박찬욱은 대한민국 출신입니다.
추가 질문: 기생충의 감독은 누구인가요?
중간 답변: 기생충의 감독은 봉준호입니다.
추가 질문: 봉준호는 어느 나라 출신인가요?
중간 답변: 봉준호는 대한민국 출신입니다.
최종 답변은: 예
""",
    },
]
examples

[{'question': '스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?',
  'answer': '이 질문에 추가 질문이 필요한가요: 예.\n추가 질문: 스티브 잡스는 몇 살에 사망했나요?\n중간 답변: 스티브 잡스는 56세에 사망했습니다.\n추가 질문: 아인슈타인은 몇 살에 사망했나요?\n중간 답변: 아인슈타인은 76세에 사망했습니다.\n최종 답변은: 아인슈타인\n'},
 {'question': '네이버의 창립자는 언제 태어났나요?',
  'answer': '이 질문에 추가 질문이 필요한가요: 예.\n추가 질문: 네이버의 창립자는 누구인가요?\n중간 답변: 네이버는 이해진에 의해 창립되었습니다.\n추가 질문: 이해진은 언제 태어났나요?\n중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.\n최종 답변은: 1967년 6월 22일\n'},
 {'question': '율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?',
  'answer': '이 질문에 추가 질문이 필요한가요: 예.\n추가 질문: 율곡 이이의 어머니는 누구인가요?\n중간 답변: 율곡 이이의 어머니는 신사임당입니다.\n추가 질문: 신사임당은 언제 태어났나요?\n중간 답변: 신사임당은 1504년에 태어났습니다.\n추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?\n중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.\n최종 답변은: 연산군\n'},
 {'question': '올드보이와 기생충의 감독이 같은 나라 출신인가요?',
  'answer': '이 질문에 추가 질문이 필요한가요: 예.\n추가 질문: 올드보이의 감독은 누구인가요?\n중간 답변: 올드보이의 감독은 박찬욱입니다.\n추가 질문: 박찬욱은 어느 나라 출신인가요?\n중간 답변: 박찬욱은 대한민국 출신입니다.\n추가 질문: 기생충의 감독은 누구인가요?\n중간 답변: 기생충의 감독은 봉준호입니다.\n추가 질문: 봉준호는 어느 나라 출신인가요?\n중간 답변: 봉준호는 대한민국 출신입니다.\n최종 답변은:

In [5]:
example_prompt = PromptTemplate.from_template(
    "Question:\n{question}\nAnswer:\n{answer}"
)
example_prompt

PromptTemplate(input_variables=['answer', 'question'], input_types={}, partial_variables={}, template='Question:\n{question}\nAnswer:\n{answer}')

In [6]:
print(example_prompt.format(**examples[0]))

Question:
스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인



FewShotPrompt 만들어보기

In [7]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt, # PromptTemplate로 만듬
    suffix="Question:\n{question}\nAnswer:", # 질문 부분
    input_variables=["question"],
)
prompt

FewShotPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, examples=[{'question': '스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?', 'answer': '이 질문에 추가 질문이 필요한가요: 예.\n추가 질문: 스티브 잡스는 몇 살에 사망했나요?\n중간 답변: 스티브 잡스는 56세에 사망했습니다.\n추가 질문: 아인슈타인은 몇 살에 사망했나요?\n중간 답변: 아인슈타인은 76세에 사망했습니다.\n최종 답변은: 아인슈타인\n'}, {'question': '네이버의 창립자는 언제 태어났나요?', 'answer': '이 질문에 추가 질문이 필요한가요: 예.\n추가 질문: 네이버의 창립자는 누구인가요?\n중간 답변: 네이버는 이해진에 의해 창립되었습니다.\n추가 질문: 이해진은 언제 태어났나요?\n중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.\n최종 답변은: 1967년 6월 22일\n'}, {'question': '율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?', 'answer': '이 질문에 추가 질문이 필요한가요: 예.\n추가 질문: 율곡 이이의 어머니는 누구인가요?\n중간 답변: 율곡 이이의 어머니는 신사임당입니다.\n추가 질문: 신사임당은 언제 태어났나요?\n중간 답변: 신사임당은 1504년에 태어났습니다.\n추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?\n중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.\n최종 답변은: 연산군\n'}, {'question': '올드보이와 기생충의 감독이 같은 나라 출신인가요?', 'answer': '이 질문에 추가 질문이 필요한가요: 예.\n추가 질문: 올드보이의 감독은 누구인가요?\n중간 답변: 올드보이의 감독은 박찬욱입니다.\n추가 질문: 박찬욱은 어느 나라 출신인가요?\n중간 답변: 박찬욱은 대한민국 출신입니다.\n추가 질문: 기생충의 감독은 누구인

In [8]:
question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"
final_prompt = prompt.format(question=question)
print(final_prompt)

Question:
스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인


Question:
네이버의 창립자는 언제 태어났나요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일


Question:
율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 율곡 이이의 어머니는 누구인가요?
중간 답변: 율곡 이이의 어머니는 신사임당입니다.
추가 질문: 신사임당은 언제 태어났나요?
중간 답변: 신사임당은 1504년에 태어났습니다.
추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
최종 답변은: 연산군


Question:
올드보이와 기생충의 감독이 같은 나라 출신인가요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 올드보이의 감독은 누구인가요?
중간 답변: 올드보이의 감독은 박찬욱입니다.
추가 질문: 박찬욱은 어느 나라 출신인가요?
중간 답변: 박찬욱은 대한민국 출신입니다.
추가 질문: 기생충의 감독은 누구인가요?
중간 답변: 기생충의 감독은 봉준호입니다.
추가 질문: 봉준호는 어느 나라 출신인가요?
중간 답변: 봉준호는 대한민국 출신입니다.
최종 답변은: 예


Question:
Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?
Answer:


llm에 입력해보기

In [9]:
answer = model.stream(final_prompt)
stream_response(answer)

이 질문에 추가 질문이 필요한가요: 예.  
추가 질문: Google은 몇 년에 창립되었나요?  
중간 답변: Google은 1998년에 창립되었습니다.  
추가 질문: Bill Gates는 몇 년에 태어났나요?  
중간 답변: Bill Gates는 1955년에 태어났습니다.  
추가 질문: 1998년에는 Bill Gates가 몇 살이었나요?  
중간 답변: 1998년에는 Bill Gates가 43세였습니다.  
최종 답변은: 43세

chain에 입력해보기

In [10]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

# chain 생성
chain = prompt | model | StrOutputParser()

# 결과 출력
answer = chain.stream(
    {"question": "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"}
)
stream_response(answer)

이 질문에 추가 질문이 필요한가요: 예.  
추가 질문: Google은 몇 년에 창립되었나요?  
중간 답변: Google은 1998년에 창립되었습니다.  
추가 질문: Bill Gates는 몇 년에 태어났나요?  
중간 답변: Bill Gates는 1955년에 태어났습니다.  
추가 질문: 1998년에는 Bill Gates가 몇 살이었나요?  
중간 답변: 1998년에는 Bill Gates가 43세였습니다.  
최종 답변은: 43세